In [1]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

engine = create_engine("sqlite:///resources/database.sqlite")

In [2]:
def selected_period_info(selected_date_1, selected_date_2):
    # Return the MDA and MTR prices for a given date.

    days_table = engine.execute(
                                   f"SELECT DISTINCT\
                                      DATE(fecha) AS dias\
                                      FROM prices_table\
                                    WHERE dias BETWEEN DATE('{selected_date_1}') AND DATE('{selected_date_2}')"
    )

    period_days = []
    for day in days_table:
        period_days.append(day[0])
    
    table_1 = engine.execute( 
                                "SELECT\
                                   sistema,\
                                   DATE(fecha) AS fecha,\
                                   zona_carga,\
                                   AVG(CAST(precio_mda AS DECIMAL)) AS precio_mda_prom,\
                                   AVG(CAST(precio_energia_mda AS DECIMAL)) AS precio_energia_mda_prom,\
                                   AVG(CAST(precio_perdida_mda AS DECIMAL)) AS precio_perdida_mda_prom,\
                                   AVG(CAST(precio_congestion_mda AS DECIMAL)) AS precio_congestion_mda_prom,\
                                   AVG(CAST(precio_mtr AS DECIMAL)) AS precio_mtr_prom,\
                                   AVG(CAST(precio_energia_mtr AS DECIMAL)) AS precio_energia_mtr_prom,\
                                   AVG(CAST(precio_perdida_mtr AS DECIMAL)) AS precio_perdida_mtr_prom,\
                                   AVG(CAST(precio_congestion_mtr AS DECIMAL)) AS precio_congestion_mtr_prom\
                                 FROM prices_table\
                                 WHERE fecha BETWEEN DATE('" + selected_date_1 + "') AND DATE('" + selected_date_2 + "')\
                                 GROUP BY fecha, zona_carga, sistema\
                                 ORDER BY sistema, fecha"
                                )
    zonas = []
    for row in table_1:
        if row[2] not in zonas:
            zonas.append(row[2])

    precios = ['precio_mda_promedio', 'precio_energia_mda_promedio', 'precio_perdida_mda_promedio', 'precio_congestion_mda_promedio',
               'precio_mtr_promedio', 'precio_energia_mtr_promedio', 'precio_perdida_mtr_promedio', 'precio_congestion_mtr_promedio']

    period_data = {
        'period_days':period_days,
        'dates':{
            day:{
                zone:{
                    precio:[] for precio in precios
                } for zone in zonas
            } for day in period_days
            }
    }
    
    table_1 = engine.execute( 
                                "SELECT\
                                   sistema,\
                                   DATE(fecha) AS fecha,\
                                   zona_carga,\
                                   AVG(CAST(precio_mda AS DECIMAL)) AS precio_mda_prom,\
                                   AVG(CAST(precio_energia_mda AS DECIMAL)) AS precio_energia_mda_prom,\
                                   AVG(CAST(precio_perdida_mda AS DECIMAL)) AS precio_perdida_mda_prom,\
                                   AVG(CAST(precio_congestion_mda AS DECIMAL)) AS precio_congestion_mda_prom,\
                                   AVG(CAST(precio_mtr AS DECIMAL)) AS precio_mtr_prom,\
                                   AVG(CAST(precio_energia_mtr AS DECIMAL)) AS precio_energia_mtr_prom,\
                                   AVG(CAST(precio_perdida_mtr AS DECIMAL)) AS precio_perdida_mtr_prom,\
                                   AVG(CAST(precio_congestion_mtr AS DECIMAL)) AS precio_congestion_mtr_prom\
                                 FROM prices_table\
                                 WHERE fecha BETWEEN DATE('" + selected_date_1 + "') AND DATE('" + selected_date_2 + "')\
                                 GROUP BY fecha, zona_carga, sistema\
                                 ORDER BY sistema, fecha"
                                )
    for row in table_1:
        period_data['dates'][str(row[1])][str(row[2])]['precio_mda_promedio'].append(row[3])
        period_data['dates'][row[1]][row[2]]['precio_energia_mda_promedio'].append(row[4])
        period_data['dates'][row[1]][row[2]]['precio_perdida_mda_promedio'].append(row[5])
        period_data['dates'][row[1]][row[2]]['precio_congestion_mda_promedio'].append(row[6])
        period_data['dates'][row[1]][row[2]]['precio_mtr_promedio'].append(row[7])
        period_data['dates'][row[1]][row[2]]['precio_energia_mtr_promedio'].append(row[8])
        period_data['dates'][row[1]][row[2]]['precio_perdida_mtr_promedio'].append(row[9])
        period_data['dates'][row[1]][row[2]]['precio_congestion_mtr_promedio'].append(row[10])
        
    return period_data

In [3]:
def selected_day_info(date):

    precios = ['precio_mda_promedio', 'precio_energia_mda_promedio', 'precio_perdida_mda_promedio', 'precio_congestion_mda_promedio',
               'precio_mtr_promedio', 'precio_energia_mtr_promedio', 'precio_perdida_mtr_promedio', 'precio_congestion_mtr_promedio']

    day_data = {
                date: {
                    precio:[] for precio in precios
                }
            }

    table_2 = engine.execute( "SELECT\
                                   DATE(fecha) AS fecha,\
                                   AVG(CAST(precio_mda AS DECIMAL)) AS precio_mda_prom,\
                                   AVG(CAST(precio_energia_mda AS DECIMAL)) AS precio_energia_mda_prom,\
                                   AVG(CAST(precio_perdida_mda AS DECIMAL)) AS precio_perdida_mda_prom,\
                                   AVG(CAST(precio_congestion_mda AS DECIMAL)) AS precio_congestion_mda_prom,\
                                   AVG(CAST(precio_mtr AS DECIMAL)) AS precio_mtr_prom,\
                                   AVG(CAST(precio_energia_mtr AS DECIMAL)) AS precio_energia_mtr_prom,\
                                   AVG(CAST(precio_perdida_mtr AS DECIMAL)) AS precio_perdida_mtr_prom,\
                                   AVG(CAST(precio_congestion_mtr AS DECIMAL)) AS precio_congestion_mtr_prom\
                                 FROM prices_table\
                                 WHERE fecha = DATE('" + date + "')"
    )

    for row in table_2:
        day_data[date]['precio_mda_promedio'].append(row[1])
        day_data[date]['precio_energia_mda_promedio'].append(row[2])
        day_data[date]['precio_perdida_mda_promedio'].append(row[3])
        day_data[date]['precio_congestion_mda_promedio'].append(row[4])
        day_data[date]['precio_mtr_promedio'].append(row[5])
        day_data[date]['precio_energia_mtr_promedio'].append(row[6])
        day_data[date]['precio_perdida_mtr_promedio'].append(row[7])
        day_data[date]['precio_congestion_mtr_promedio'].append(row[8])

    return day_data

In [10]:
def selected_period_days(zone):

    precios = ['precio_mda_promedio', 'precio_mtr_promedio']

    zone_data = {
                'zone':zone,
                 '2018-01-10': {
                     precio:[] for precio in precios
                 }
             }
    table_3 = engine.execute(
                                "SELECT\
                                   zona_carga,\
                                   fecha,\
                                   AVG(CAST(precio_mda AS DECIMAL)) AS precio_mda_prom,\
                                   AVG(CAST(precio_mtr AS DECIMAL)) AS precio_mtr_prom\
                                FROM prices_table\
                                WHERE DATE(fecha) BETWEEN DATE ('2018-01-10', '-6 days') AND DATE('2018-01-10') AND zona_carga = '" + zone + "'\
                                GROUP BY zona_carga, fecha"
    )

    for row in table_3:
        zone_data['2018-01-10']['precio_mda_promedio'].append(row[2])
        zone_data['2018-01-10']['precio_mtr_promedio'].append(row[3])

    return zone_data

In [11]:
selected_period_days('ENSENADA')

{'zone': 'ENSENADA',
 '2018-01-10': {'precio_mda_promedio': [968.8979166666667,
   846.3954166666667,
   631.7366666666668,
   362.5895833333334,
   467.94583333333327,
   472.4266666666667,
   325.7070833333333],
  'precio_mtr_promedio': [900.4754166666665,
   931.6912500000002,
   684.5329166666666,
   504.9370833333333,
   531.0470833333334,
   553.51625,
   544.7045833333333]}}